# Goal: Given a taxi trip, predict the travel time!

Useful data analysis:
https://www.julienphalip.com/blog/kaggle-competition-report-ecml-pkdd-2015-taxi/

paper on how to develop models:
https://arxiv.org/abs/1508.00021


### General Info

TRIP_ID: (String) - An unique identifier for each trip; \
CALL_TYPE: (char) - Category of the ride. It may contain one of three possible values: \
‘A’ if this trip was dispatched from the central \
‘B’ if this trip was demanded directly to a taxi driver on a specific stand \
‘C’ otherwise (i.e. a trip demanded on a random street \
ORIGIN_CALL: (integer) - An unique identifier for the phone number to call the taxi. It identifies the trip’s customer \ 
if CALL_TYPE=’A’. Otherwise, it assumes a NULL value; \
ORIGIN_STAND: (integer) - An unique identifier for the taxi stand. It identifies the starting point of the trip if CALL_TYPE=’B’. Otherwise, it assumes a NULL value; \
TAXI_ID: (integer) - An unique identifier for the taxi that performed each trip; \
TIMESTAMP: (integer) - Unix Timestamp (in seconds). It identifies the trip’s start; \
DAYTYPE: (char) - It identifies the daytype of the trip’s start. It assumes one of three possible values: \
‘A’ if this trip started on a normal day or weekend \
‘B’ if this trip started on a holiday or other special day \
‘C’ if the trip started on a day before a type-B day \
MISSING_DATA: (Boolean) - It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing \
POLYLINE: (String): - A list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ]). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. The coordinates were recorded every 15 seconds during the trip. The first item represents the starting point and the last item corresponds to the destination. \
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

#https://www.julienphalip.com/blog/kaggle-competition-report-ecml-pkdd-2015-taxi/
import pickle
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# from matplotlib.ticker import  MultipleLocator, FormatStrFormatterfrom sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
raw_train = pd.read_csv("train.csv")

In [4]:
#missing data 
missing = raw_train.query('MISSING_DATA == True')

### Remove the missing data!

In [5]:
train = raw_train.query('MISSING_DATA == False')
train

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


In [6]:
train_rows = train.shape[0] 

In [7]:
#splitting data 80-10-10 (training-validating-testing)

perm = torch.randperm(train_rows) #return indices from 0 to 1710660
eighty = int(train_rows * 0.80)
ten = int(train_rows * 0.10)
trainning_idx = perm[:eighty]
validate_idx = perm[eighty:eighty + ten]
test_idx = perm[eighty + ten:]
# print(trainning_idx.shape[0] + validate_idx.shape[0] + test_idx.shape[0] == train_rows)

In [8]:
# #you can also split this by doing
# train, validation, train_labels, validation_labels = train_test_split(train, train_labels, test_size=0.02)
# validation, test, validation_labels, test_labels = train_test_split(validation, validation_labels, test_size=0.5)

Polyline: (longitude, latitude) update for every 15 seconds

# Data cleaning... 
Figure out how much each trip ID lasted! \
Using code from discussion

In [21]:
## Polyline_cnt = number of 15 second points
train["POLYLINE_CT"] = train["POLYLINE"].apply(lambda x : max(x.count("[") - 1, 0))
#warning appears, but still works!

/tmp/ipykernel_302/169734621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["POLYLINE_CT"] = train["POLYLINE"].apply(lambda x : max(x.count("[") - 1, 0))


In [19]:
#add trip duration
train["TRIP_DUR_SEC"] = train["POLYLINE_CT"].apply(lambda x : x * 15)

/tmp/ipykernel_302/204856974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["TRIP_DUR_SEC"] = train["POLYLINE_CT"].apply(lambda x : x * 15)


In [23]:
train_no_missing_poly = train[['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP', 'DAY_TYPE', 'POLYLINE_CT', 'TRIP_DUR_SEC']]
train_no_missing_poly

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE_CT,TRIP_DUR_SEC
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,23,345
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,19,285
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,65,975
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,43,645
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,29,435
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,32,480
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,30,450
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,0,0
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,62,930
